In [1]:
!pip install aitextgen
!pip install tensorflow

In [2]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Mirror Cake As Made By Duff Goldman

1 cup glucose
¾ cup sweetened condensed milk
1 ½ cups granulated sugar
⅔ cup water
2 cups white chocolate chip
0.7 oz gelatin
0.7 oz food coloring
1 tablespoon titanium dioxide
1 tablespoon glitter


1. Bloom gelatin by mixing gelatin into cold water and letting it sit for at least 5 minutes.
2. Heat granulated sugar, corn syrup, and water on a stove top to a boil.
3. In a bowl, combine the white chocolate chips, sweetened condensed milk, and solidified gelatin.
4. Pour the hot sugar mixture over the chocolate, condensed milk, and gelatin. Cover with a cloth until chocolate is soft.
5. Mix thoroughly until all chocolate is completely melted.
6. Add desired color and titanium dioxide to the hot mixture. Mix until there are no air bubbles.
7. Let mixture cool to 90°F (33°C), then pour over the frozen cake.
8. Note, this recipe works best on a cake covered in a frozen mousse, but will also work with a cake covered in frozen buttercream frosting.
9. Enj

In [3]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe_fixed.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [4]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, to_gpu=True, config=config, tf_gpt2="355M")


C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching checkpoint: 1.05Mit [00:00, 1.05Git/s]                 
Fetching hparams.json: 1.05Mit [00:00, 1.05Git/s]                 
Fetching model.ckpt.data-00000-of-00001: 1.42Git [11:21, 2.08Mit/s]                           
Fetching model.ckpt.index: 1.05Mit [00:00, 1.05Git/s]                  
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.38Mit/s]                 
Converting TensorFlow checkpoint from d:\Temp Files\Ai Camp Cooking Project\aitextgen\355M
Loading TF weight model/h0/attn/c_attn/b with shape [3072]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 1024, 3072]
Loading TF weight model/h0/attn/c_proj/b with shape [1024]
Loading TF w

Save PyTorch model to aitextgen/pytorch_model.bin
Save configuration file to aitextgen/config.json


In [16]:
# #Ai training disabled to prevent accidental overrides.


# file_name = "recipe_fixed.csv"
# ai.save("aitextgen.model")
# train_tokenizer(file_name)
# data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
# ai.train(data, batch_size=32, num_steps=64000, generate_every=1000, save_every=5000)
# ai.generate(prompt="steak")

# #Save the model
# ai.save("aitextgen.model")
#     if __name__ == '__main__':
#             freeze_support()

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.


  0%|          | 0/4961 [00:00<?, ?it/s]

MisconfigurationException: `Trainer(strategy='ddp_spawn')` or `Trainer(accelerator='ddp_spawn')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: Trainer(strategy=None|dp|tpu_spawn). In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.

In [7]:
# Load the trained model
ai = aitextgen(model_folder="trained_model", tokenizer_file="aitextgen.tokenizer.json")
ai.generate(prompt="cherry pie")

C:\Users\flash\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


cherry pie filling over the cheesecake. Using a torch, dash the tops of the pastry cream into the cheesecake using a kn-mold.
14. Chill the muffins in the freezer for 5 minutes or until they hard topping is firm.
15.
15. Enjoy!
Brown Storelave lef Allow Storeassure Peaution Pautions Paut Cookies





16. In a can Nook Natme Rice mixed Bre is mostcoated carrot top sirlide Shou can bean

St peach




16. Instant drizzle the can firlide Upautrilled Mix the can gelichoundshurpawauté mochohound the apple





Bread
2 can firlide the sparrothort peach
1 tablespoon of anylide bistact with the app
